In [1]:
from fastai.vision.all import *
from fastai.vision.widgets import *

In [2]:
from PIL import Image 
AmazingCredit = Image.open(r"./AmazingCredit.png")
GoodCredit = Image.open(r"./GoodCredit.png")
BadCredit = Image.open(r"./BadCredit.png")

import ipywidgets as widget

AmazingCredit =  widget.HTML('<div id="img" style="float:left">'
                    '<img style="vertical-align:middle" src="./AmazingCredit.png" alt="">'                       
                    '</div>')

GoodCredit =  widget.HTML('<div id="img" style="float:left">'
                    '<img style="vertical-align:middle" src="./GoodCredit.png" alt="">'                       
                    '</div>')

BadCredit =  widget.HTML('<div id="img" style="float:left">'
                    '<img style="vertical-align:middle" src="./BadCredit.png" alt="">'                       
                    '</div>')

In [3]:
# graph_call = open("thing.sh", "w")
# address = "0x00d5201c4def4d1a199c63e07eac53f43252a0ea"
# variable = '''curl 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2' --data-binary '{"query":"query MyQuery { users(where: {id: \\"''' + address + '''\\"}) { borrowHistory { borrowRate  borrowRateMode  amount  id  timestamp } liquidationCallHistory { collateralAmount  id  timestamp  } } } ","variables":{}}' > call_data.txt'''
# graph_call.write(r"{}".format(variable))
# graph_call.close()

In [4]:
# os.system("bash thing.sh")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1907    0  1663  100   244  13304   1952 --:--:-- --:--:-- --:--:-- 15256


0

In [5]:
data_loading_bay = {"avg_borrowRate" : [], "avg_amount_borrowed" : [], "num_borrows" : [], "num_prev_defaults" : [], "default_this_time" : []}

In [6]:
def eth_to_dict(input_address):
    graph_call = open("thing.sh", "w")
    address = input_address
    variable = '''curl 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2' --data-binary '{"query":"query MyQuery { users(where: {id: \\"''' + address + '''\\"}) { borrowHistory { borrowRate  borrowRateMode  amount  id  timestamp } liquidationCallHistory { collateralAmount  id  timestamp  } } } ","variables":{}}' > call_data.txt -s'''
    graph_call.write(r"{}".format(variable))
    graph_call.close()

    os.system("bash thing.sh")

    data_loading_bay = {"avg_borrowRate" : [], "avg_amount_borrowed" : [], "num_borrows" : [], "num_prev_defaults" : [], "default_this_time" : []}

    with open("call_data.txt") as json_file:
      all_data = json.load(json_file)
      data = all_data["data"]
      for user in data["users"]:
        if user["borrowHistory"] != []:
          num_borrows = 0
          total_amount = 0
          total_borrowRate = 0
          num_prev_defaults = 0
          for borrow in user["borrowHistory"]:
            num_borrows += 1
            total_amount += int(borrow["amount"])
            total_borrowRate += int(borrow["borrowRate"])
            last_borrow_id = borrow["id"]
          if user["liquidationCallHistory"] != []:
            for default in user["liquidationCallHistory"]:
              num_prev_defaults = 0
              for _ in user["liquidationCallHistory"]:
                num_prev_defaults += 1
              # if default["id"] == last_borrow_id:
                default_this_time = 1
                # num_prev_defaults -= 1
              # else:
              #   default_this_time = 0
          else:
            prev_defaults = 0
            default_this_time = 0

          data_loading_bay["avg_borrowRate"].append(total_borrowRate / num_borrows)
          data_loading_bay["avg_amount_borrowed"].append(total_amount / num_borrows)
          data_loading_bay["num_borrows"].append(num_borrows)
          data_loading_bay["num_prev_defaults"].append(num_prev_defaults)
          data_loading_bay["default_this_time"].append(default_this_time)
    return data_loading_bay

# CredAAVE

A tool allowing you to check the credit score of accounts using AAVE.  
(Insert why this is a problem, explain credit score)  
Built using GraphQL, AAVE, and PyTorch

[Video Demo](https://loom.com/)  
[GitHub](https://github.com/)  

-------

In [32]:
path = Path()
learn_inf = load_learner(path/'testing3.pkl', cpu=True)
txt_input = widgets.Text()
out_pl = widgets.Output()
lbl_pred = widgets.Label()

In [33]:
# pred_df = pd.DataFrame(data_loading_bay)
# pred_df_true = pred_df.squeeze()

In [34]:
# cred = float(learn_inf.predict(pred_df_true)[2])
# cred

In [52]:
def display_credit(cred):
    cred = 1 - cred
    cred -= 0.5
    cred *= 6
    cred = 1 / (1 + math.exp(-cred))
    if cred >= 0.9:
        print("Amazing credit score!")
        display(AmazingCredit)
    elif cred >=0.7:
        print("Good credit score!")
        display(GoodCredit)
    else:
        print ("Bad credit score!")
        display(BadCredit)

In [53]:

# if cred >= 0.9:
#     AmazingCredit
# elif cred >=0.7:
#     GoodCredit
# else:
#     BadCredit

In [54]:
def on_text_change(change):
    lbl_pred.value = ''
    out_pl.clear_output()
    data_loading_bay = eth_to_dict(change)
    pred_df = pd.DataFrame(data_loading_bay)
    pred_df_true = pred_df.squeeze()
    cred = float(learn_inf.predict(pred_df_true)[2])
    return display_credit(cred)
    # lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

#def on_text_change(change):
#    lbl_pred.value = ''
#    out_pl.clear_output()
#    pred,pred_idx,probs = learn_inf.predict(img)
#    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [55]:
# txt_input.observe(on_text_change, names=['data'])

In [56]:
display(VBox([widgets.Label('Input a valid Ethereum address:'), txt_input, out_pl, lbl_pred]))

In [57]:
button = widgets.Button(description='Run It!')

In [61]:
out = widgets.Output()
def on_button_clicked(_):
      # "linking function with output"
      # i = 0
      with out:
          # if i != 1:
          #   pass
          # else:
          # print(txt_input)
          # print(str(txt_input)[12:-2])
        # clear_output()
        out.clear_output()
        on_text_change(str(txt_input)[12:-2])
# linking button and function together using a button's method
button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([button, out])